In [1]:
import web_scraper, ArticleScraper

In [3]:
from mongoengine import *
import datetime

connect('mongoengine_2', host='localhost', port=27017)

class TaggedText(EmbeddedDocument):
    content = StringField()
    text_tag = StringField()
    
class Author(Document):
#     author = StringField(required=True)
    name = StringField()
    
    
class Article(Document):
    title = StringField(required=True)
    contents = ListField(EmbeddedDocumentField(TaggedText))
    author = ReferenceField(Author, reverse_delete_rule=CASCADE)
    posted_date = DateTimeField(default=datetime.datetime.now)
    published_status = BooleanField(default=False)
    source_url = URLField()
    source = StringField()

    credits = StringField()
    tags = ListField(StringField(max_length=30))
    img_urls = ListField(URLField())

In [4]:
import time

In [ ]:
with open('cleaned_data_url.txt', 'r') as f:
    data = f.readlines()
 
    for line in data:
#         words = line.split()
#         print (line)
#         print(type(line))

        print('Processing URL...')
        current_page = web_scraper.soupified_request(line)
        
        tag_text_list = []

        tag_text_list.append(['title', ArticleScraper.get_article_title(current_page)])

        body = list(current_page.find('div', {'class' : 'field-item even first last'}))

        ArticleScraper.tag_text_list_gen(body, tag_text_list)

        title = tag_text_list[0][1]
        
        contentss = []

        for i in range(1, len(tag_text_list)):

            tagg = tag_text_list[i][0]
            textt = tag_text_list[i][1]

            contentss.append(TaggedText(content=textt, text_tag=tagg))
            
        article_1 = Article(title=title, contents=contentss)
        
        article_1.save()
        
        print('\nCompleted URL...\n')
        time.sleep(11)
        print('\nSleeping!\n')

        